In [104]:

# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [105]:
import numpy as np

class Neuron():
    def __init__(self, input_dim):
        self.weights = np.random.rand(input_dim)
        self.bias = np.random.random()
        print ("Neuron bias: ", self.bias)

In [106]:
# inherit from base class Layer
class FCLayer(Layer):
    # input_dim = number of input neurons
    # output_dim = number of output neurons
    def __init__(self, input_dim, output_dim):
        self.neurons = [Neuron(input_dim) for _ in range(output_dim)]
        self.weights = self.get_weights()
        self.bias = self.get_bias()

    def get_weights(self):
        weights = np.vstack([neuron.weights for neuron in self.neurons])
        return weights.T

    def get_bias(self):
        bias = np.vstack([neuron.bias for neuron in self.neurons])
        return bias.T

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # dBias = output_error

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [107]:


# inherit from base class Layer
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [108]:

# activation function and its derivative
def tanh(x):
    return np.tanh(x);

def tanh_prime(x):
    return 1-np.tanh(x)**2;

In [109]:

# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

In [110]:

class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [111]:
# training data
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# network
net = Network()
net.add(FCLayer(2, 3))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(3, 1))
net.add(ActivationLayer(tanh, tanh_prime))

# train
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=1000, learning_rate=0.1)

# test
out = net.predict(x_train)
print(out)

------------------------layer-----------------------
input_dim = 2 output_dim = 3
Neuron bias:  0.11978563952364585
Neuron bias:  0.4781370852996627
Neuron bias:  0.9798922515205193
---------------------
Weights neuron:
 [[0.12042404 0.78651655 0.25640383]
 [0.26517845 0.42965621 0.7147434 ]] <class 'numpy.ndarray'>
Bias neuron :
 [[0.11978564 0.47813709 0.97989225]] <class 'numpy.ndarray'>
--------------------
------------------------layer-----------------------
input_dim = 3 output_dim = 1
Neuron bias:  0.6405960892952952
---------------------
Weights neuron:
 [[0.2496566 ]
 [0.54575443]
 [0.15163673]] <class 'numpy.ndarray'>
Bias neuron :
 [[0.64059609]] <class 'numpy.ndarray'>
--------------------
epoch 1/1000   error=0.355166
epoch 2/1000   error=0.323731
epoch 3/1000   error=0.302671
epoch 4/1000   error=0.295734
epoch 5/1000   error=0.294910
epoch 6/1000   error=0.294666
epoch 7/1000   error=0.294036
epoch 8/1000   error=0.293146
epoch 9/1000   error=0.292124
epoch 10/1000   err

epoch 191/1000   error=0.164746
epoch 192/1000   error=0.163145
epoch 193/1000   error=0.161474
epoch 194/1000   error=0.159734
epoch 195/1000   error=0.157921
epoch 196/1000   error=0.156032
epoch 197/1000   error=0.154065
epoch 198/1000   error=0.152014
epoch 199/1000   error=0.149875
epoch 200/1000   error=0.147642
epoch 201/1000   error=0.145307
epoch 202/1000   error=0.142864
epoch 203/1000   error=0.140305
epoch 204/1000   error=0.137623
epoch 205/1000   error=0.134809
epoch 206/1000   error=0.131859
epoch 207/1000   error=0.128765
epoch 208/1000   error=0.125526
epoch 209/1000   error=0.122137
epoch 210/1000   error=0.118601
epoch 211/1000   error=0.114918
epoch 212/1000   error=0.111096
epoch 213/1000   error=0.107141
epoch 214/1000   error=0.103067
epoch 215/1000   error=0.098888
epoch 216/1000   error=0.094623
epoch 217/1000   error=0.090293
epoch 218/1000   error=0.085924
epoch 219/1000   error=0.081544
epoch 220/1000   error=0.077182
epoch 221/1000   error=0.072869
epoch 22